Gabor ML - Problem 10

Outline:

Functions:
-Generate linearly separably data
-Generate non-linearly separably data
-Perceptron algorithm

Uses:
-test algorithm on different dimensions of data and types of data

In [2]:
#Libraries

import pandas as pd
import numpy as np
import random
import matplotlib

In [13]:
#Functions

#Generate linearly separable data
def generate_ls_data(a, d, n):
    #a is a number in range [0,1]
    #d is dimension of sample data
    #n is number of sample points

    

    #x is a list of n numpy arrays
    #y is a list of n corresponding {-1,1}

    #determine y values with probability .5
    probability_y_equals_1 = .5
    y = [1 if random.random() >= probability_y_equals_1 else -1 for _ in range(n)]
    x = []

    #creates list of x iid sample vectors based on problem conditions
    for i in range(n):
        x_i = np.zeros(d)
        for j in range(d):
            #first element based on a
            if j == 0:
                if y[i] == 1:
                    x_i[j] = random.uniform(-1,-a)
                else:
                    x_i[j] = random.uniform(a,1)
            #other elements based on [-1,1]
            else:
                x_i[j] = random.uniform(-1,1)
        x.append(x_i)        

    return x, y

def generate_nls_data(m, d, n):

    #create y data
    probability_y_equals_1 = .5
    y = [1 if random.random() >= probability_y_equals_1 else -1 for _ in range(n)]

    #create non-linearly separable data per problem description
    mean = np.zeros(d)
    mean[0] = m
    cov = np.identity(d)
    
    x_arr = np.random.multivariate_normal(mean, cov, n)
    x = [x_arr[i] for i in range(n)]

    return x, y


In [ ]:
a = .2
d = 2
n = 4

x, y = generate_ls_data(a, d, n)